In [1]:
import gradio as gr
import requests


def uploader(IP, Update_type, firmware=None, filesystem=None):

    if Update_type not in ["firmware", "filesystem"]:
        return "Invalid upload type. Choose 'firmware' or 'filesystem'."

    if Update_type == "firmware":
        file_name = "firmware.bin"
        with open(firmware, "rb") as f:
            file_data = f.read()
    elif Update_type == "filesystem":
        file_name = "filesystem.bin"
        with open(filesystem, "rb") as f:
            file_data = f.read()

    if not file_data:
        return f"Error: {Update_type} file not provided."

    url = f"http://{IP}/otaupdate"

    response = requests.post(url, files={file_name: file_data}, headers={
                             "Content-Type": "application/octet-stream", "X-OTA-PASSWORD": "admin"})

    if response.status_code == 200:
        return f"{Update_type.capitalize()} uploaded successfully"
    else:
        try:
            error_message = response.text
        except UnicodeDecodeError:
            error_message = "Error decoding response content"

        return f"Error uploading {Update_type}: {error_message}"

iface = gr.Interface(
    fn=uploader,
    inputs=[
        gr.Textbox(label="IP Address", lines=1, type="text"),
        gr.Dropdown(["firmware", "filesystem"],
                    label="Update Type", interactive=True),
        gr.File(label="Firmware", type="filepath"),
        gr.File(label="Filesystem", type="filepath")
    ],
    outputs="text",
    title="FIRMWARE UPLOADER",
    description="Upload firmware and filesystem to the device"
)

if __name__ == "__main__":
    iface.launch(show_api=False, width=900, height=1000, show_error=True)

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.
